## vedio to image(15frame)

In [ ]:
import cv2
import os
import time
from tqdm import tqdm

# 동영상 파일 경로
video_path = 'E:/5.spring_of_loa/SOL/Lost Ark/Video/Raid/striker_valtan_gate_01.mp4'

# 이미지 저장 경로
output_folder = 'frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 동영상 읽기
cap = cv2.VideoCapture(video_path)

# 전체 프레임 수 확인
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 프레임 카운터 초기화
frame_count = 0

# 저장할 이미지 파일 이름 초기화
saved_frame_count = 0

# 시작 시간 기록
start_time = time.time()

# tqdm으로 진행 상태 표시
for _ in tqdm(range(0, total_frames, 4), desc="Processing frames", unit="frame"):
    # 4프레임마다 동영상에서 프레임 읽기
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
    ret, frame = cap.read()
    
    if not ret:
        print("동영상 읽기가 끝났습니다.")
        break

    # 이미지 저장
    cv2.imwrite(f'{output_folder}/{saved_frame_count}.jpg', frame)

    saved_frame_count += 1  # 1씩 증가
    frame_count += 4  # 4프레임씩 건너뛰기

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자가 종료키를 눌렀습니다.")
        break

# 종료 시간 기록 및 소요 시간 계산
end_time = time.time()
elapsed_time = end_time - start_time
print(f"전체 처리 시간: {elapsed_time:.2f} 초")

cap.release()
cv2.destroyAllWindows()


## 이미지 전처리

### RGB

In [1]:
# 551/554/544 번 노랑색
# 544 = 데미지 크기 큰거 (숫자 겹침)
# 551 = 데미지 크기 비슷한거 (숫자 겹침)
# 554 = 데미지 하나짜리

import cv2
import numpy as np
import matplotlib.pyplot as plt

# 이미지 불러오기
image_path = 'frames/544.jpg'
image = cv2.imread(image_path)

# 특정 색상 범위 설정 (BGR 순서)
# 각 채널을 더 넓게 확장된 범위 설정
upper_bound = np.array([90, 255, 255])  # 확장된 범위
lower_bound = np.array([30, 150, 120])  # 확장된 범위

# 이미지 크기를 700x700으로 줄임
resize_img = cv2.resize(image, (720, 405))

# 이미지 크기를 1920x1080으로 다시 늘림
resize_img_1 = cv2.resize(resize_img, (1920, 1080))

# 특정 색상 범위에 해당하는 부분을 이진화하여 마스크 생성
mask = cv2.inRange(resize_img_1, lower_bound, upper_bound)

# 특정 색상 부분만 추출
result = cv2.bitwise_and(image, image, mask=mask)

# 결과 이미지 출력
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


# JPG로 결과 이미지 저장
cv2.imwrite('result.jpg', result)

True

### cmyk

In [35]:
# # 544 = 데미지 크기 큰거 (숫자 겹침)
# # 551 = 데미지 크기 비슷한거 (숫자 겹침)
# # 554 = 데미지 하나짜리
import cv2
import numpy as np

# RGB to CMYK 변환 함수
def rgb_to_cmyk(rgb_image):
    # 이미지를 0~1 사이로 정규화
    rgb_image = rgb_image.astype(np.float32) / 255.0
    
    r = rgb_image[:,:,2]
    g = rgb_image[:,:,1]
    b = rgb_image[:,:,0]
    
    c = 1 - r
    m = 1 - g
    y = 1 - b
    
    k = np.minimum(c, np.minimum(m, y))
    
    c = (c - k) / (1 - k + 1e-10)  # 0으로 나누는 오류를 피하기 위해 작은 값 추가
    m = (m - k) / (1 - k + 1e-10)
    y = (y - k) / (1 - k + 1e-10)
    
    # K 값은 그대로 사용
    k = k
    
    # CMYK 이미지 생성 (0~100% 범위로 변환)
    cmyk_image = np.stack([c, m, y, k], axis=-1) * 100
    cmyk_image = cmyk_image.astype(np.uint8)
    
    return cmyk_image

# 이미지 불러오기
image_path = 'frames/544.jpg'
image = cv2.imread(image_path)

# RGB 이미지를 CMYK로 변환
cmyk_image = rgb_to_cmyk(image)

# CMYK 색상 범위 설정 (0~100%)
# [C, M, Y, K]
# upper_bound_cmyk = np.array([100, 100, 100, 100])  # 상한값
# lower_bound_cmyk = np.array([0, 0, 0, 0])  # 하한값
upper_bound_cmyk = np.array([0, 30, 100, 10])  # 상한값
lower_bound_cmyk = np.array([0, 0, 60, 0])  # 하한값

# cmyk_image.jpg로 저장
cv2.imwrite('cmyk_image.jpg', cmyk_image)
            
# CMYK 이미지를 700x700으로 줄임
resize_cmyk_img = cv2.resize(cmyk_image, (720, 405))

# CMYK 이미지를 1920x1080으로 다시 늘림
resize_cmyk_img_1 = cv2.resize(resize_cmyk_img, (1920, 1080))

# 특정 색상 범위에 해당하는 부분을 이진화하여 마스크 생성
mask_cmyk = cv2.inRange(resize_cmyk_img_1, lower_bound_cmyk, upper_bound_cmyk)

# 특정 색상 부분만 추출
result_cmyk = cv2.bitwise_and(image, image, mask=mask_cmyk)

# 결과 이미지 출력
cv2.imshow('Result CMYK', result_cmyk)
cv2.waitKey(0)
cv2.destroyAllWindows()

# JPG로 결과 이미지 저장
cv2.imwrite('result_cmyk.jpg', result_cmyk)


True

## OCR 인식 확인

In [33]:
import cv2
import pytesseract
import numpy as np

# 이미지 불러오기
# image_path = 'result.jpg'
image_path = 'result_cmyk.jpg'

image = cv2.imread(image_path)

# 이미지를 그레이스케일로 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 이진화
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 노이즈 제거를 위한 미디언 블러 적용
median = cv2.medianBlur(thresh, 3)

# 이미지 확장
kernel = np.ones((2, 2), np.uint8)
dilation = cv2.dilate(median, kernel, iterations=1)

# Tesseract OCR을 사용하여 이미지에서 숫자만 추출
text = pytesseract.image_to_string(dilation, config='--psm 6 outputbase digits')
print("Recognized Numbers:", text)

# 이미지 새 창에 띄우기
cv2.imshow('Processed Image', dilation)
cv2.waitKey(0)
cv2.destroyAllWindows()



Recognized Numbers: 27748081



In [ ]:
import cv2
import pytesseract
import numpy as np

# 이미지 불러오기
# image_path = 'result.jpg'
image_path = 'result_cmyk.jpg'

image = cv2.imread(image_path)

# 이미지를 그레이스케일로 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 가우시안 노이즈 추가
noise = np.random.normal(0, 10, gray.shape)  # 표준편차 값을 10으로 조정
noisy_gray = np.clip(gray + noise, 0, 255).astype(np.uint8)

# 이진화
_, thresh = cv2.threshold(noisy_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# 노이즈 제거를 위한 미디언 블러 적용
median = cv2.medianBlur(thresh, 3)

# 이미지 확장
kernel = np.ones((2, 2), np.uint8)
dilation = cv2.dilate(median, kernel, iterations=1)

# Tesseract OCR을 사용하여 이미지에서 숫자만 추출
text = pytesseract.image_to_string(dilation, config='--psm 6 outputbase digits')
print("Recognized Numbers:", text)

# 이미지 새 창에 띄우기
cv2.imshow('Processed Image', dilation)
cv2.waitKey(0)
cv2.destroyAllWindows()
